In [6]:
%load_ext autoreload
%autoreload 2

In [54]:
import csv, functools, operator,os
import pandas as pd
import numpy as np 
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
data_dir = '/Users/jaodand/Dropbox/PostDoc/Jobs:Fellowships/TDI/JHU/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # to read all csv files
data_files = functools.reduce(operator.add, [glob(pattern) for pattern in [
    "%s/*.csv*"%data_dir]])

In [27]:
df = pd.read_csv('%s'%data_files[0], sep=",")

In [64]:
Hdrs = list(df)
df_Ind = df[df['Country_Region'].str.contains("India")]

In [58]:
x_idx = np.arange(len(df_Ind))+1
plt.hist(x_idx,df_Ind['Deaths'])

ValueError: `bins` must increase monotonically, when an array

In [60]:
df_Ind['Deaths'], 


(247       62
 248     7108
 249       56
 250     1045
 251     1397
 252      317
 253     3371
 254        2
 255    10536
 256      739
 257     4306
 258     2905
 259      934
 260     1883
 261     1030
 262    12090
 263     3072
 264      127
 265        0
 266     3606
 267    49521
 268      355
 269      139
 270        8
 271       79
 272     1873
 273      633
 274     5341
 275     2696
 276      127
 277    12122
 278     1544
 279      385
 280        0
 281     8364
 282     1509
 283     9712
 Name: Deaths, dtype: int64,
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37]))